### Import libraries

In [1]:
import requests
import json
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize
import time

from lxml import objectify

from bibliopixel import *
from bibliopixel.drivers.LPD8806 import *
from bibliopixel import LEDStrip
import bibliopixel.colors as colors

### Setting up LED

In [2]:
LedsPerSide = 8
numLeds= LedsPerSide*4*2 ##x/side * 4 sides * 2 levels
driver=DriverLPD8806(numLeds)
led=LEDStrip(driver)

ERROR - spi_driver_base - Unable to import spidev. Please install. pip install spidev
ERROR:BiblioPixel:Unable to import spidev. Please install. pip install spidev


ImportError: Unable to import spidev. Please install. pip install spidev

###Extracting data from xml files

In [3]:
path = 'cc_customer_11.xml'
xml = objectify.parse(open(path))
root = xml.getroot()

In [4]:
#Meter readings in root.getchildren[7]
obj_top = root.getchildren()[7].getchildren()[4].getchildren()[0].getchildren()
length = len(obj_top)
length

105408

In [5]:
df = pd.DataFrame(columns=('duration', 'time_stamp', 'value'))

In [6]:
for i in range(1,length): #takes a while to run #element 0 is summary
    obj = obj_top[i].getchildren()
    row = dict(zip(['duration', 'time_stamp', 'value'], [np.int(obj[0].getchildren()[0]), np.int(obj[0].getchildren()[1]), np.int(obj[1])]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)

In [7]:
(df['value'].mean(), df['value'].min(), df['value'].max()) # 'value' is in watt-hrs

(318232.83326534292, 85751.0, 527377.0)

### Adding timestamp and grouping

In [8]:
start_time = pd.to_datetime('2011-12-31 20:10') 
## Got this from the sheet: 
## Link: http://services.greenbuttondata.org/sample-data/cc_customer_11.xml

In [41]:
df['timestamp'] = pd.date_range(start_time, periods=105407, freq='5min') 
#Creates time stamp to be added to each entry in the right format

df['year'] = pd.DatetimeIndex(df['timestamp']).year
df['month'] = pd.DatetimeIndex(df['timestamp']).month
df['day'] = pd.DatetimeIndex(df['timestamp']).day

df_monthly = df.groupby(['month','year'])
df_monthly_total = df_monthly['value'].sum() #Monthly total usage
df_daily = df.groupby(['year','month','day'])
df_daily_total = df_daily['value'].sum()


In [38]:
new_start_time = pd.to_datetime('2012-1-1')
new_end_time = pd.to_datetime('2012-12-31')
all_days_2012 = pd.date_range(new_start_time, new_end_time, freq='D')
d = str(all_days_2012[0].year)+'-'+str(all_days_2012[0].month)+'-'+str(all_days_2012[0].day)
df_daily_value = df_daily['value'].get_group(all_days_2012[0].date).mean()
    

KeyError: <built-in method date of Timestamp object at 0x116795150>

In [42]:
df_daily_total

year  month  day
2011  12     31      4528002
2012  1      1      26401996
             2      59138126
             3      83857793
             4      89380782
             5      91529382
             6      87702373
             7      76825226
             8      65986796
             9      84503772
             10     86354413
             11     89445878
             12     92382816
             13     89715917
             14     73735570
             15     64330596
             16     82438528
             17     89637550
             18     93241970
             19     90947095
             20     91122673
             21     78034516
             22     66194447
             23     87008085
             24     89558297
             25     92634710
             26     93196149
             27     91119559
             28     77480512
             29     71423815
                      ...   
      12     2      72957918
             3      91677508
             4      962392

In [23]:
year = 2012
df_by_month = pd.DataFrame(columns=('Energy_mean_monthly', 'Year', 'Month'))
df_by_day = pd.DataFrame(columns=('Energy_mean_daily', 'Year' , 'Month', 'Day'))

for i in range (1,13): 
    row = dict(zip(['Energy_mean_monthly', 'Year', 'Month'],[df_monthly['value'].get_group((year, i)).mean(),year,i]))
    row_s = pd.Series(row)
    row_s.name = i
    df_new = df_new.append(row_s)
    
    


In [24]:
df_new

,Energy_mean,Month,Year
1,283286.791331,1,2012
2,307228.791547,2,2012
3,308425.411850,3,2012
4,294379.706481,4,2012
5,323817.569220,5,2012
6,359240.309954,6,2012
7,375794.022961,7,2012
8,370624.506384,8,2012
9,325972.901852,9,2012
10,298914.979839,10,2012


### Generating color scale

In [28]:
red = 0
green = 255
stepSize = 25
color_gn_rd = []
color_gn_rd.append((green, red, 0)) ## the lights are GRB format

In [69]:
while(red < 255): ## start with green and increase red
    red += stepSize;
    if(red > 245):
        red = 255; 
    color_gn_rd.append((green,red,0))

while(green > 0): ## start with red + green and decrease green
    green -= stepSize;
    if(green < 6):
        green = 0; 
    color_gn_rd.append((green,red,0)); 
    
len(color_gn_rd)

21

### Defining Flashing modes

In [67]:
intensity = np.arange(0,1.1,0.1)
reversed(intensity)
color = (50,100,110)
color_new = (color[0]*.9, color[1]*0.9,color[2]*0.9)
color_new

(45.0, 90.0, 99.0)

In [68]:
def led_set(start_position, numLEDs, color): ## Fills the colors
    led.fill(color, start=start_position,end=start_position+numLEDs)
    led.update()
    return

def pulse(start_position, numLEDs, color):
    ## Step-up intensity by 10% increments, then step down by the same every 0.1 seconds. total time = 4 sec
    
    intensity = np.arange(0,1.1,0.1) 

    for i in intensity:
        color_new = (color[0]*i,color[1]*i,color[3]*i) 
        # There is probably a more elegant way to do this.. 
        led.fill(color_new, start=start_position,end=start_position+numLEDs)
        led.update()
        time.sleep(0.2)

    for i in reversed(intensity):
        color_new = (color[0]*i,color[1]*i,color[3]*i) 
        led.fill(color_new, start=start_position,end=start_position+numLEDs)
        led.update()
        time.sleep(0.2)        

In [70]:
print ('Starting Display')
print ('Press \'control + c\' to stop' )


Starting Display
Press 'control + c' to stop


### Display 1: Last 10 day average on top, current usage on bottom

In [ ]:
i = 11 
last_index = len(df_daily_total)
last_10day_average = []
color_top = []
color_bottom = []
while (i < last_index):
    last_10day_average.append(df_daily_total[i-11:i-1].mean())
    